In [1]:
import pandas as pd
import json

In [2]:
us_states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado",
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho",
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana",
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota",
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada",
    "New Hampshire", "New Jersey", "New Mexico", "New York",
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon",
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]

In [3]:
# Create a empty Dataframe to concat to
temp_df = pd.DataFrame({})

# Loop through all 50 US states
for state in us_states:

    # Convert states to file names
    state_filename = f"{state.replace(' ', '_')}.json"

    # Use filename to open and store file in data_list
    with open('./state_data/Alabama.json', 'r') as file:
        data_list = json.load(file)

    # Loop through data_list and convert each json data in dataframe
    for item in data_list:
        for data in item.values():
            data = pd.json_normalize(data)
            data['state'] = state
            temp_df = pd.concat([temp_df, data], ignore_index=True)
temp_df


,lat,lon,tz,date,units,cloud_cover.afternoon,humidity.afternoon,precipitation.total,temperature.min,temperature.max,temperature.afternoon,temperature.night,temperature.evening,temperature.morning,pressure.afternoon,wind.max.speed,wind.max.direction,state
0,33.037792,-86.649186,-06:00,1980-01-16,standard,48.04,68.16,10.48,280.66,289.16,288.80,281.80,285.91,281.63,1020.82,3.95,130.55,Alabama
1,33.037792,-86.649186,-06:00,1980-01-31,standard,100.00,66.01,6.60,268.88,284.62,273.01,284.62,271.40,272.82,1025.64,5.83,349.34,Alabama
2,33.037792,-86.649186,-06:00,1980-02-15,standard,93.51,65.63,9.75,277.85,291.40,289.87,280.20,289.03,278.17,1016.84,4.51,201.23,Alabama
3,33.037792,-86.649186,-06:00,1980-03-01,standard,100.00,95.13,20.22,268.44,284.49,284.11,281.11,270.19,278.34,1013.76,5.61,331.84,Alabama
4,33.037792,-86.649186,-06:00,1980-03-16,standard,97.91,60.15,0.41,285.17,293.68,291.91,285.93,291.91,285.58,1022.63,6.52,163.68,Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53395,33.037792,-86.649186,-05:00,2024-04-30,standard,100.00,77.00,0.00,291.32,300.19,296.74,293.86,299.51,291.88,1016.00,4.12,160.00,Wyoming
53396,33.037792,-86.649186,-05:00,2024-05-15,standard,75.00,66.00,0.00,291.06,299.77,298.44,292.93,298.97,291.06,1010.00,4.63,300.00,Wyoming
53397,33.037792,-86.649186,-05:00,2024-05-30,standard,40.00,55.00,0.00,291.88,301.62,298.40,294.05,301.14,291.88,1021.00,2.24,16.00,Wyoming
53398,33.037792,-86.649186,-05:00,2024-06-14,standard,0.00,50.00,0.00,294.02,307.85,306.55,296.61,306.64,294.02,1012.00,2.06,130.00,Wyoming


In [4]:
temp_df.to_csv('../data/state_weather_data.csv', index=False)

In [5]:
temp_df['month_day'] = temp_df['date'].apply(lambda x: x[5:])

all_dates = []

for date in temp_df['month_day'].value_counts().keys():
    all_dates.append(date)

# Create a function to map dates to quarters
def assign_quarter(date):
    month = int(date.split('-')[0])  # Extract month
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

# Apply the function to assign quarters
temp_df['quarter'] = temp_df['month_day'].apply(assign_quarter)

temp_df['quarter'].value_counts()

quarter
Q2    15150
Q3    13200
Q4    13200
Q1    11850
Name: count, dtype: int64

In [7]:
test_df = temp_df.copy()

test_df['year'] = test_df['date'].apply(lambda x: x[:4])

test_df['year']

0        1980
1        1980
2        1980
3        1980
4        1980
         ... 
53395    2024
53396    2024
53397    2024
53398    2024
53399    2024
Name: year, Length: 53400, dtype: object

In [8]:
# Get rid of all object columns except the columns we use to group (state, quarter)
test_df = test_df.drop(columns=['lat', 'lon', 'tz', 'date', 'units', 'month_day'])
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53400 entries, 0 to 53399
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cloud_cover.afternoon  53400 non-null  float64
 1   humidity.afternoon     53400 non-null  float64
 2   precipitation.total    53400 non-null  float64
 3   temperature.min        53400 non-null  float64
 4   temperature.max        53400 non-null  float64
 5   temperature.afternoon  53400 non-null  float64
 6   temperature.night      53400 non-null  float64
 7   temperature.evening    53400 non-null  float64
 8   temperature.morning    53400 non-null  float64
 9   pressure.afternoon     53400 non-null  float64
 10  wind.max.speed         53400 non-null  float64
 11  wind.max.direction     53400 non-null  float64
 12  state                  53400 non-null  object 
 13  quarter                53400 non-null  object 
 14  year                   53400 non-null  object 
dtypes:

In [9]:
grouped_df = test_df.groupby(['state', 'quarter', 'year']).mean().reset_index()

grouped_df

,state,quarter,year,cloud_cover.afternoon,humidity.afternoon,precipitation.total,temperature.min,temperature.max,temperature.afternoon,temperature.night,temperature.evening,temperature.morning,pressure.afternoon,wind.max.speed,wind.max.direction
0,Alabama,Q1,1980,75.735000,67.498333,7.910000,277.080000,289.098333,286.361667,283.133333,282.753333,279.715000,1018.783333,5.151667,242.193333
1,Alabama,Q1,1981,47.528000,48.048000,0.092000,275.494000,287.446000,285.976000,278.474000,284.816000,276.208000,1021.624000,4.818000,221.754000
2,Alabama,Q1,1982,75.294000,73.532000,6.532000,278.410000,290.300000,288.494000,284.152000,284.844000,284.428000,1017.372000,4.676000,222.838000
3,Alabama,Q1,1983,72.450000,59.328000,2.584000,276.706000,289.176000,286.708000,279.208000,284.578000,277.054000,1013.456000,3.550000,186.388000
4,Alabama,Q1,1984,33.130000,52.585000,0.225000,275.708333,287.823333,286.011667,278.455000,284.608333,276.465000,1022.005000,3.146667,260.798333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8895,Wyoming,Q4,2019,63.500000,59.000000,7.396667,279.496667,290.935000,287.466667,285.026667,284.413333,282.855000,1022.500000,6.433333,178.333333
8896,Wyoming,Q4,2020,56.500000,71.333333,0.821667,284.273333,294.061667,292.153333,287.133333,290.480000,285.850000,1017.833333,3.423333,226.833333
8897,Wyoming,Q4,2021,25.666667,57.500000,1.635000,282.271667,293.128333,291.653333,284.741667,288.083333,283.108333,1012.333333,3.168333,156.833333
8898,Wyoming,Q4,2022,61.666667,70.166667,0.191667,281.198333,291.550000,289.428333,286.263333,286.550000,284.453333,1017.666667,4.066667,198.833333


In [11]:
grouped_df.columns

Index(['state', 'quarter', 'year', 'cloud_cover.afternoon',
       'humidity.afternoon', 'precipitation.total', 'temperature.min',
       'temperature.max', 'temperature.afternoon', 'temperature.night',
       'temperature.evening', 'temperature.morning', 'pressure.afternoon',
       'wind.max.speed', 'wind.max.direction'],
      dtype='object')

In [15]:
pivoted_df = grouped_df.pivot_table(
    index=['state', 'year'],      # We keep 'state' and 'year' as the index
    columns='quarter',            # Pivot on the 'quarter' column
    values=[                      # Specify all columns to pivot (excluding 'state', 'year', 'quarter')
        'cloud_cover.afternoon', 'humidity.afternoon', 'precipitation.total',
        'temperature.min', 'temperature.max', 'temperature.afternoon',
        'temperature.night', 'temperature.evening', 'temperature.morning',
        'pressure.afternoon', 'wind.max.speed', 'wind.max.direction'
    ]
)

# Flatten the multi-level column index
pivoted_df.columns = [f'{col[1]}_{col[0]}' for col in pivoted_df.columns]

# Reset index to turn 'state' and 'year' back into columns
pivoted_df.reset_index(inplace=True)

# Display the result
pivoted_df

,state,year,Q1_cloud_cover.afternoon,Q2_cloud_cover.afternoon,Q3_cloud_cover.afternoon,Q4_cloud_cover.afternoon,Q1_humidity.afternoon,Q2_humidity.afternoon,Q3_humidity.afternoon,Q4_humidity.afternoon,...,Q3_temperature.night,Q4_temperature.night,Q1_wind.max.direction,Q2_wind.max.direction,Q3_wind.max.direction,Q4_wind.max.direction,Q1_wind.max.speed,Q2_wind.max.speed,Q3_wind.max.speed,Q4_wind.max.speed
0,Alabama,1980,75.735000,44.480000,40.675000,52.151667,67.498333,55.990000,51.900000,50.010000,...,297.231667,280.063333,242.193333,198.683333,188.051667,185.386667,5.151667,3.636667,3.155000,4.641667
1,Alabama,1981,47.528000,65.207143,48.771667,35.768333,48.048000,45.842857,58.491667,64.810000,...,296.306667,283.935000,221.754000,173.021429,183.100000,99.535000,4.818000,3.534286,2.866667,3.656667
2,Alabama,1982,75.294000,52.621429,56.115000,71.518333,73.532000,61.824286,65.210000,69.565000,...,295.173333,287.206667,222.838000,185.530000,131.945000,193.718333,4.676000,3.221429,3.026667,5.816667
3,Alabama,1983,72.450000,53.022857,41.711667,49.433333,59.328000,57.222857,59.598333,64.446667,...,295.193333,282.078333,186.388000,235.011429,151.050000,231.078333,3.550000,4.607143,3.500000,4.715000
4,Alabama,1984,33.130000,61.528333,52.196667,58.903333,52.585000,54.216667,67.193333,60.461667,...,295.041667,284.205000,260.798333,127.723333,127.528333,162.966667,3.146667,4.155000,2.831667,3.945000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,Wyoming,2020,43.000000,26.166667,64.333333,56.500000,60.000000,54.833333,78.333333,71.333333,...,296.256667,287.133333,209.500000,261.666667,195.000000,226.833333,4.845000,5.228333,4.651667,3.423333
2246,Wyoming,2021,84.000000,11.571429,45.500000,25.666667,77.000000,46.428571,68.000000,57.500000,...,295.845000,284.741667,214.400000,176.142857,210.000000,156.833333,5.394000,4.154286,2.305000,3.168333
2247,Wyoming,2022,19.600000,44.857143,19.500000,61.666667,52.200000,69.285714,56.166667,70.166667,...,293.805000,286.263333,237.200000,218.571429,113.833333,198.833333,2.758000,3.504286,3.460000,4.066667
2248,Wyoming,2023,99.800000,34.142857,29.166667,49.500000,82.600000,61.714286,70.500000,63.166667,...,296.251667,283.233333,236.000000,226.571429,258.000000,135.000000,5.146000,4.822857,4.593333,3.171667


In [16]:
# Create a new feature 'is_future' to mark rows with missing (future) data
pivoted_df['is_future'] = pivoted_df.apply(lambda row: 1 if row.isnull().any() else 0, axis=1)

# Replace NaN values with -1 to indicate missing data
pivoted_df.fillna(-1, inplace=True)

# Now we have two ways of representing future data:
# 1. -1 in the weather-related columns
# 2. The 'is_future' column marking rows with future data
pivoted_df

,state,year,Q1_cloud_cover.afternoon,Q2_cloud_cover.afternoon,Q3_cloud_cover.afternoon,Q4_cloud_cover.afternoon,Q1_humidity.afternoon,Q2_humidity.afternoon,Q3_humidity.afternoon,Q4_humidity.afternoon,...,Q4_temperature.night,Q1_wind.max.direction,Q2_wind.max.direction,Q3_wind.max.direction,Q4_wind.max.direction,Q1_wind.max.speed,Q2_wind.max.speed,Q3_wind.max.speed,Q4_wind.max.speed,is_future
0,Alabama,1980,75.735000,44.480000,40.675000,52.151667,67.498333,55.990000,51.900000,50.010000,...,280.063333,242.193333,198.683333,188.051667,185.386667,5.151667,3.636667,3.155000,4.641667,0
1,Alabama,1981,47.528000,65.207143,48.771667,35.768333,48.048000,45.842857,58.491667,64.810000,...,283.935000,221.754000,173.021429,183.100000,99.535000,4.818000,3.534286,2.866667,3.656667,0
2,Alabama,1982,75.294000,52.621429,56.115000,71.518333,73.532000,61.824286,65.210000,69.565000,...,287.206667,222.838000,185.530000,131.945000,193.718333,4.676000,3.221429,3.026667,5.816667,0
3,Alabama,1983,72.450000,53.022857,41.711667,49.433333,59.328000,57.222857,59.598333,64.446667,...,282.078333,186.388000,235.011429,151.050000,231.078333,3.550000,4.607143,3.500000,4.715000,0
4,Alabama,1984,33.130000,61.528333,52.196667,58.903333,52.585000,54.216667,67.193333,60.461667,...,284.205000,260.798333,127.723333,127.528333,162.966667,3.146667,4.155000,2.831667,3.945000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,Wyoming,2020,43.000000,26.166667,64.333333,56.500000,60.000000,54.833333,78.333333,71.333333,...,287.133333,209.500000,261.666667,195.000000,226.833333,4.845000,5.228333,4.651667,3.423333,0
2246,Wyoming,2021,84.000000,11.571429,45.500000,25.666667,77.000000,46.428571,68.000000,57.500000,...,284.741667,214.400000,176.142857,210.000000,156.833333,5.394000,4.154286,2.305000,3.168333,0
2247,Wyoming,2022,19.600000,44.857143,19.500000,61.666667,52.200000,69.285714,56.166667,70.166667,...,286.263333,237.200000,218.571429,113.833333,198.833333,2.758000,3.504286,3.460000,4.066667,0
2248,Wyoming,2023,99.800000,34.142857,29.166667,49.500000,82.600000,61.714286,70.500000,63.166667,...,283.233333,236.000000,226.571429,258.000000,135.000000,5.146000,4.822857,4.593333,3.171667,0


In [17]:
pivoted_df.to_csv('../data/clean_weather_data.csv', index=False)